# Conversation Chain

In [1]:
# pip install --upgrade langchain langchain-openai

In [2]:
# !pip install langchain-core langchain-community 

In [3]:
from langchain_openai.chat_models.base import ChatOpenAI
api_key = ""
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0, api_key=api_key)

## Conversation Buffer Memory

ConversationBufferMemory usage is straightforward. It simply keeps the entire conversation in the buffer memory up to the allowed max limit (e.g. 4096 for gpt-3.5-turbo, 8192 for gpt-4). The main downside, however, is the cost. Each request is sending the aggregation to the API. Since the charge is based on token usages, the cost can quickly add up, especially if you are sending requests to an AI platform. Additionally, there can be added latency given the sheer size of the text that’s being passed back and forth.



In [4]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [5]:
conversation_buffer = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory(),
    verbose=True
)

In [6]:
conversation_buffer("Hi There!")

/Users/c100-122/opt/anaconda3/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi There!
AI:

> Finished chain.


{'input': 'Hi There!',
 'history': '',
 'response': 'Hello! How can I assist you today?'}

In [7]:
conversation_buffer("What are some popular ones?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi There!
AI: Hello! How can I assist you today?
Human: What are some popular ones?
AI:

> Finished chain.


{'input': 'What are some popular ones?',
 'history': 'Human: Hi There!\nAI: Hello! How can I assist you today?',
 'response': 'Popular what? Can you please provide more context so I can better assist you?'}

## Conversation Buffer Window Memory
Given that most conversations will not need context from several messages before, we also have the option of also using ConversationBufferWindowMemory. With this option we can simply control the buffer with a window of the last k messages.

This enables us to control how much context is sent in the request, yet still maintain adequate recent conversational history.

In [8]:
from langchain.memory.buffer_window import ConversationBufferWindowMemory

In [9]:
conversation_buffer_window = ConversationChain(
    llm=llm,
    memory=ConversationBufferWindowMemory(k=3),
    verbose=True
)

In [10]:
conversation_buffer_window("Hi There!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi There!
AI:

> Finished chain.


{'input': 'Hi There!',
 'history': '',
 'response': 'Hello! How can I assist you today?'}

In [11]:
conversation_buffer_window("What are some popular ones?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi There!
AI: Hello! How can I assist you today?
Human: What are some popular ones?
AI:

> Finished chain.


{'input': 'What are some popular ones?',
 'history': 'Human: Hi There!\nAI: Hello! How can I assist you today?',
 'response': 'Popular what? Can you please provide more context so I can better assist you?'}

## Conversation Summary Memory
ConversationSummaryMemory does not keep the entire history in memory like ConversationBufferMemory. Nor does it maintain a window. Rather, the ConversationSummaryMemory continually summarizes the conversation as it’s happening, and maintains it so we have context from the beginning of the conversation.

In [12]:
from langchain.memory.summary import ConversationSummaryMemory
from langchain_community.llms.openai import OpenAI 

In [13]:
conversation_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryMemory(llm=llm),
    verbose=True
)

In [14]:
"""
Final question in the ongoing conversation.
"""

conversation_summary("What are some popular ones?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What are some popular ones?
AI:

> Finished chain.


{'input': 'What are some popular ones?',
 'history': '',
 'response': 'There are many popular AI assistants out there, such as Siri by Apple, Alexa by Amazon, Google Assistant, and Cortana by Microsoft. These assistants are widely used for tasks like setting reminders, answering questions, playing music, and controlling smart home devices. Each assistant has its own unique features and capabilities, making them popular choices for users looking to simplify their daily tasks.'}

## Conversation Summary Buffer Memory
ConversationSummaryBufferMemory gives more nuanced control to maintain the most recent messages while summarizing the previous ones. There is the ability to flush into the summary based on a token count and keep the rest in the buffer. This could be helpful when it is important to have exact context of the last few messages, while still maintaining summary of older interactions within the conversation.

In [15]:
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory

In [16]:
conversation_summary_buf = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(llm=llm, max_token_limit=2000),
    verbose=True
    )

## Conversation Token Buffer Memory
ConversationTokenBufferMemory simply maintains the buffer by the token count and does not maintain. There is no summarizing with this approach . In fact, it is similar to ConversationBufferWindowMemory, except instead of flushing based on the last number of messages (k), flushing is based on the number of tokens.

In [18]:
from langchain.memory.token_buffer import ConversationTokenBufferMemory

In [19]:
conversation_token_buffer = ConversationChain(
    llm=llm,
    memory=ConversationTokenBufferMemory(llm=llm, max_token_limit=500, return_messages=True),
    verbose=True
)